In [1]:
import platform
import sys
import datetime
import netCDF4
import numpy as np
import importlib
import time
from tqdm import tqdm

base_FP = '/home/subin/data'
cpuserver_data_FP = '/home/subin/cpuserver_data'
sys.path.append(base_FP + '/python_modules')

import develop_HydroAI.HydroAI.TC_like as hTCL
import develop_HydroAI.HydroAI.Vectorization as hVec
import matplotlib.pyplot as plt
importlib.reload(hTCL);
importlib.reload(hVec);

import warnings
warnings.filterwarnings("ignore")
# Define your directory where to save nc files
nc_save_dir = cpuserver_data_FP + '/extracted_nc'

In [2]:
import numpy as np

def TCA_vec(X, Y, Z, nod_th=30, corr_th=0):
    # 0. Check for NaNs and ensure that any NaNs are consistent across datasets
    combined_nan_mask = np.isnan(X) | np.isnan(Y) | np.isnan(Z)
    X[combined_nan_mask] = np.nan
    Y[combined_nan_mask] = np.nan
    Z[combined_nan_mask] = np.nan

    # Remove completely NaN slices (if data is 3D)
    valid_mask = ~np.isnan(X).all(axis=2) & ~np.isnan(Y).all(axis=2) & ~np.isnan(Z).all(axis=2)
    X = X[valid_mask]
    Y = Y[valid_mask]
    Z = Z[valid_mask]

    # 1. Flatten the data to 2D arrays if necessary
    X_flat = X.reshape(-1, X.shape[-1])
    Y_flat = Y.reshape(-1, Y.shape[-1])
    Z_flat = Z.reshape(-1, Z.shape[-1])

    # Remove rows with NaNs
    valid_rows = ~np.isnan(X_flat).any(axis=1) & ~np.isnan(Y_flat).any(axis=1) & ~np.isnan(Z_flat).any(axis=1)
    X_flat = X_flat[valid_rows]
    Y_flat = Y_flat[valid_rows]
    Z_flat = Z_flat[valid_rows]

    # Combine the data into a single array
    data = np.stack((X_flat, Y_flat, Z_flat), axis=1)  # Shape: (N, 3, T)

    # Initialize arrays to store results
    N = data.shape[0]
    errVar_ETC = np.zeros((N, 3))
    rho2_ETC = np.zeros((N, 3))
    SNR = np.zeros((N, 3))
    SNRdb = np.zeros((N, 3))

    # Iterate over each time series (row)
    for i in range(N):
        y = data[i, :, :].T  # Shape: (T, 3)

        # Check that there are enough valid observations
        if y.shape[0] < nod_th:
            continue  # Skip if not enough data

        # Remove any remaining NaNs
        if np.isnan(y).any():
            continue  # Skip if there are NaNs

        # Compute covariance matrix
        Q_hat = np.cov(y, rowvar=False)

        # Ensure covariance matrix is full rank
        if np.linalg.matrix_rank(Q_hat) < 3:
            continue  # Skip if covariance matrix is singular

        # Calculate correlation coefficients
        try:
            rho_ETC = np.zeros(3)
            rho_ETC[0] = np.sqrt(Q_hat[0,1]*Q_hat[0,2]/(Q_hat[0,0]*Q_hat[1,2]))
            rho_ETC[1] = np.sign(Q_hat[0,2]*Q_hat[1,2]) * np.sqrt(Q_hat[0,1]*Q_hat[1,2]/(Q_hat[1,1]*Q_hat[0,2]))
            rho_ETC[2] = np.sign(Q_hat[0,1]*Q_hat[1,2]) * np.sqrt(Q_hat[0,2]*Q_hat[1,2]/(Q_hat[2,2]*Q_hat[0,1]))

            rho2_ETC[i, :] = rho_ETC**2

            # Calculate error variances
            errVar_ETC[i, 0] = Q_hat[0,0] - Q_hat[0,1]*Q_hat[0,2]/Q_hat[1,2]
            errVar_ETC[i, 1] = Q_hat[1,1] - Q_hat[0,1]*Q_hat[1,2]/Q_hat[0,2]
            errVar_ETC[i, 2] = Q_hat[2,2] - Q_hat[0,2]*Q_hat[1,2]/Q_hat[0,1]

            # Calculate SNR
            SNR[i, :] = (rho2_ETC[i, :]) / (1 - rho2_ETC[i, :])

            # Calculate SNR in dB
            SNRdb[i, :] = 10 * np.log10(SNR[i, :])

        except Exception as e:
            # Handle any mathematical errors (e.g., division by zero)
            continue

    # Prepare outputs
    VAR_err = {
        'x': errVar_ETC[:, 0],
        'y': errVar_ETC[:, 1],
        'z': errVar_ETC[:, 2]
    }
    SNR = {
        'x': SNR[:, 0],
        'y': SNR[:, 1],
        'z': SNR[:, 2]
    }
    SNRdb = {
        'x': SNRdb[:, 0],
        'y': SNRdb[:, 1],
        'z': SNRdb[:, 2]
    }
    R = {
        'x': rho2_ETC[:, 0],
        'y': rho2_ETC[:, 1],
        'z': rho2_ETC[:, 2]
    }
    fMSE = {
        'x': 1 - rho2_ETC[:, 0],
        'y': 1 - rho2_ETC[:, 1],
        'z': 1 - rho2_ETC[:, 2]
    }

    # 3. Set the flags
    # Flag on non-scaled data (not necessary here as we didn't calculate corrXY)
    # flags on correlation coefficients
    condition_corr = (R['x'] < corr_th) | (R['y'] < corr_th) | (R['z'] < corr_th)  # flag 1
    # flag on valid number of observations
    condition_n_valid = np.sum(~np.isnan(X_flat) & ~np.isnan(Y_flat) & ~np.isnan(Z_flat), axis=1) < nod_th
    # flags on fMSE
    condition_fMSE = (fMSE['x'] < 0) | (fMSE['y'] < 0) | (fMSE['z'] < 0) | (fMSE['x'] > 1) | (fMSE['y'] > 1) | (fMSE['z'] > 1)  # flag 3
    # flag on negative error variances
    condition_negative_vars_err = (errVar_ETC < 0).any(axis=1)  # flag 4

    flags = {'condition_corr': condition_corr,
            'condition_n_valid': condition_n_valid,
            'condition_fMSE': condition_fMSE,
            'condition_negative_vars_err': condition_negative_vars_err}

    return VAR_err, SNR, SNRdb, R, fMSE, flags

In [67]:
def ETC(D1, D2, D3, nod_th=30, corr_th=0):
    """
    Extended Triple Collocation (ETC) is a technique for estimating the
    variance of the noise error (errVar) and correlation coefficients (rho)
    of three measurement systems (e.g., satellite, in-situ, and model-based products)
    with respect to the unknown true value of the variable being measured
    (e.g., soil moisture, wind speed).

    INPUTS
    D1, D2, D3: Arrays of observations from the three measurement systems.
    They must be of the same length, and all NaNs must be removed or handled appropriately.

    OUTPUTS
    errVar_ETC: A list of error variances [errVar_D1, errVar_D2, errVar_D3].
    rho2_ETC: A list of squared correlation coefficients [rho2_D1, rho2_D2, rho2_D3].

    REFERENCE
    McColl, K.A., J. Vogelzang, A.G. Konings, D. Entekhabi, M. Piles, A. Stoffelen (2014).
    Extended Triple Collocation: Estimating errors and correlation coefficients with respect
    to an unknown target. Geophysical Research Letters 41:6229-6236.
    """

    # Convert inputs to numpy arrays
    D1 = np.asarray(D1)
    D2 = np.asarray(D2)
    D3 = np.asarray(D3)

    # Check that all inputs have the same length
    if not (len(D1) == len(D2) == len(D3)):
        raise ValueError('Error: Input data D1, D2, D3 must be of the same length.')

    # Combine the data into a single array
    y = np.column_stack((D1, D2, D3))  # Shape: (N, 3)

    # Remove any rows with NaNs
    if np.isnan(y).any():
        y = y[~np.isnan(y).any(axis=1)]

    # Catch errors in inputs
    if y.shape[1] != 3:
        raise ValueError('Error: Input data must result in an N x 3 array after removing NaNs.')

    if y.size == 0:
        raise ValueError('Error: No data left after removing NaNs.')

    # Check that each column has non-zero variance
    if np.var(y[:, 0]) == 0 or np.var(y[:, 1]) == 0 or np.var(y[:, 2]) == 0:
        raise ValueError('Error: The sample variance of each dataset must be non-zero.')

    # Estimate covariance matrix of the three measurement systems
    Q_hat = np.cov(y, rowvar=False)

    # Compute correlation coefficients
    rho_ETC = np.zeros(3)

    try:
        rho_ETC[0] = np.sqrt(Q_hat[0, 1] * Q_hat[0, 2] / (Q_hat[0, 0] * Q_hat[1, 2]))
        rho_ETC[1] = np.sign(Q_hat[0, 2] * Q_hat[1, 2]) * np.sqrt(Q_hat[0, 1] * Q_hat[1, 2] / (Q_hat[1, 1] * Q_hat[0, 2]))
        rho_ETC[2] = np.sign(Q_hat[0, 1] * Q_hat[1, 2]) * np.sqrt(Q_hat[0, 2] * Q_hat[1, 2] / (Q_hat[2, 2] * Q_hat[0, 1]))
    except (ZeroDivisionError, FloatingPointError, ValueError):
        raise ValueError('Error: Calculation of correlation coefficients failed due to invalid covariance values.')

    rho2_ETC = rho_ETC ** 2

    # Compute error variances
    errVar_ETC = np.zeros(3)
    errVar_ETC[0] = Q_hat[0, 0] - (Q_hat[0, 1] * Q_hat[0, 2]) / Q_hat[1, 2]
    errVar_ETC[1] = Q_hat[1, 1] - (Q_hat[0, 1] * Q_hat[1, 2]) / Q_hat[0, 2]
    errVar_ETC[2] = Q_hat[2, 2] - (Q_hat[0, 2] * Q_hat[1, 2]) / Q_hat[0, 1]

    # Check for negative error variances
    if np.any(errVar_ETC < 0):
        print('Warning: At least one calculated errVar is negative. This can happen if the sample size is too small or if one of the assumptions of ETC is violated.')

    # Check for negative squared correlation coefficients
    if np.any(rho2_ETC < 0):
        print('Warning: At least one calculated squared correlation coefficient is negative. This can happen if the sample size is too small or if one of the assumptions of ETC is violated.')

    # ======================================================
    # Computer SNR
    SNR = np.zeros(3)
    SNRdb = np.zeros(3)

    SNR[0] = (rho2_ETC[0]) / (1 - rho2_ETC[0])
    SNRdb[0] = 10 * np.log10(SNR[0])
    SNR[1] = (rho2_ETC[1]) / (1 - rho2_ETC[1])
    SNRdb[1] = 10 * np.log10(SNR[1])
    SNR[2] = (rho2_ETC[2]) / (1 - rho2_ETC[2])
    SNRdb[2] = 10 * np.log10(SNR[2])

    # Prepare outputs
    VAR_err = {
        'x': errVar_ETC[0],
        'y': errVar_ETC[1],
        'z': errVar_ETC[2]
    }
    SNR = {
        'x': SNR[0],
        'y': SNR[1],
        'z': SNR[2]
    }
    SNRdb = {
        'x': SNRdb[0],
        'y': SNRdb[1],
        'z': SNRdb[2]
    }
    R = {
        'x': rho2_ETC[0],
        'y': rho2_ETC[1],
        'z': rho2_ETC[2]
    }
    fMSE = {
        'x': 1 - rho2_ETC[0],
        'y': 1 - rho2_ETC[1],
        'z': 1 - rho2_ETC[2]
    }
    # Set the flags
    # Flag on non-scaled data (not necessary here as we didn't calculate corrXY)
    # flags on correlation coefficients
    condition_corr = (R['x'] < corr_th) | (R['y'] < corr_th) | (R['z'] < corr_th)  # flag 1
    # flag on valid number of observations
    condition_n_valid = np.sum(~np.isnan(D1) & ~np.isnan(D2) & ~np.isnan(D3), axis=0) < nod_th # flag 2 (axis=0 is due to 1D array
    # flags on fMSE
    condition_fMSE = (fMSE['x'] < 0) | (fMSE['y'] < 0) | (fMSE['z'] < 0) | (fMSE['x'] > 1) | (fMSE['y'] > 1) | (fMSE['z'] > 1)  # flag 3
    # flag on negative error variances
    condition_negative_vars_err = (errVar_ETC < 0).any(axis=0)  # flag 4 (axis=0 is due to 1D array)

    flags = {'condition_corr': condition_corr,
            'condition_n_valid': condition_n_valid,
            'condition_fMSE': condition_fMSE,
            'condition_negative_vars_err': condition_negative_vars_err}

    return VAR_err, SNR, SNRdb, R, fMSE, flags

In [3]:
import numpy as np

np.random.seed(0)  # For reproducibility

N = 1000  # Number of data points

# Generate the true signal T
T = np.random.normal(0, 1, N)  # Mean 0, standard deviation 1

# Generate errors for each dataset
e1 = np.random.normal(0, 0.5, N)  # Error variance 0.25
e2 = np.random.normal(0, 0.7, N)  # Error variance 0.49
e3 = np.random.normal(0, 0.9, N)  # Error variance 0.81

# Generate the datasets
D1 = T + e1
D2 = T + e2
D3 = T + e3

In [4]:
# Reshape data to 3D arrays with shape (1, 1, N)
X = D1.reshape(1, 1, N)
Y = D2.reshape(1, 1, N)
Z = D3.reshape(1, 1, N)

In [5]:
# Previous TCA_vec_old code
VAR_err_TCA_old, SNR_TCA_old, SNRdb_TCA_old, R_TCA_old, fMSE_TCA_old, flags_TCA_old = hTCL.TCA_vec_old(X, Y, Z)
print(np.shape(VAR_err_TCA_old['x']))
print(np.shape(flags_TCA_old['condition_n_valid']))
print(VAR_err_TCA_old['x'])

# Assume TCA_vec is already defined as per the modified code
VAR_err_TCA, SNR_TCA, SNRdb_TCA, R_TCA, fMSE_TCA, flags_TCA = hTCL.TCA_vec(X, Y, Z)
print(np.shape(VAR_err_TCA['x']))
print(VAR_err_TCA['x'])

# Assume ETC is already defined as per the provided code
errVar_ETC, SNR_ETC, SNRdb_ETC, R_ETC, fMSE_ETC, flags_ETC = hTCL.ETC(D1, D2, D3)
print(np.shape(errVar_ETC['x']))
print(errVar_ETC['x'])

(1, 1)
(1, 1)
[[0.23401675]]
(1,)
[0.23401675]
()
0.23401675296766955


In [6]:
# Previous TCA_vec_old code
errVar_TCA_old_x = VAR_err_TCA_old['x'][0, 0]  # Since the arrays have shape (1, 1)
errVar_TCA_old_y = VAR_err_TCA_old['y'][0, 0]
errVar_TCA_old_z = VAR_err_TCA_old['z'][0, 0]
errVar_TCA_old = np.array([errVar_TCA_old_x, errVar_TCA_old_y, errVar_TCA_old_z])
print(np.shape(errVar_TCA_old))

# Extract error variances from TCA_vec outputs
errVar_TCA_x = VAR_err_TCA['x'][0]  # Since the arrays have shape (1,)
errVar_TCA_y = VAR_err_TCA['y'][0]
errVar_TCA_z = VAR_err_TCA['z'][0]
# Combine into a single array for comparison
errVar_TCA = np.array([errVar_TCA_x, errVar_TCA_y, errVar_TCA_z])
print(np.shape(errVar_TCA))

# Extract error variances from ETC outputs
errVar_ETC_x = errVar_ETC['x']
errVar_ETC_y = errVar_ETC['y']
errVar_ETC_z = errVar_ETC['z']
# Combine into a single array for comparison
errVar_ETC = np.array([errVar_ETC_x, errVar_ETC_y, errVar_ETC_z])
print(np.shape(errVar_ETC))


(3,)
(3,)
(3,)


In [7]:
print("Error Variances from TCA_vec_old:")
print(f"D1: {errVar_TCA_old[0]:.4f}, D2: {errVar_TCA_old[1]:.4f}, D3: {errVar_TCA_old[2]:.4f}")

print("\nError Variances from TCA_vec:")
print(f"D1: {errVar_TCA[0]:.4f}, D2: {errVar_TCA[1]:.4f}, D3: {errVar_TCA[2]:.4f}")

print("\nError Variances from ETC:")
print(f"D1: {errVar_ETC[0]:.4f}, D2: {errVar_ETC[1]:.4f}, D3: {errVar_ETC[2]:.4f}")

Error Variances from TCA_vec_old:
D1: 0.2340, D2: 0.4550, D3: 0.7692

Error Variances from TCA_vec:
D1: 0.2340, D2: 0.4474, D3: 0.8209

Error Variances from ETC:
D1: 0.2340, D2: 0.4474, D3: 0.8209


In [8]:
print("SNR from TCA_vec_old:")
print(f"D1: {SNR_TCA_old['x'][0, 0]:.4f}, D2: {SNR_TCA_old['y'][0, 0]:.4f}, D3: {SNR_TCA_old['z'][0, 0]:.4f}")

print("\nSNR from TCA_vec:")
print(f"D1: {SNR_TCA['x'][0]:.4f}, D2: {SNR_TCA['y'][0]:.4f}, D3: {SNR_TCA['z'][0]:.4f}")

print("\nSNR from ETC:")
print(f"D1: {SNR_ETC['x']:.4f}, D2: {SNR_ETC['y']:.4f}, D3: {SNR_ETC['z']:.4f}")

SNR from TCA_vec_old:
D1: 4.0408, D2: 2.0668, D3: 1.2247

SNR from TCA_vec:
D1: 4.0408, D2: 2.0668, D3: 1.2247

SNR from ETC:
D1: 4.0408, D2: 2.0668, D3: 1.2247


In [9]:
print("SNRdb from TCA_vec_old:")
print(f"D1: {SNRdb_TCA_old['x'][0, 0]:.4f}, D2: {SNRdb_TCA_old['y'][0, 0]:.4f}, D3: {SNRdb_TCA_old['z'][0, 0]:.4f}")

print("\nSNRdb from TCA_vec:")
print(f"D1: {SNRdb_TCA['x'][0]:.4f}, D2: {SNRdb_TCA['y'][0]:.4f}, D3: {SNRdb_TCA['z'][0]:.4f}")

print("\nSNRdb from ETC:")
print(f"D1: {SNRdb_ETC['x']:.4f}, D2: {SNRdb_ETC['y']:.4f}, D3: {SNRdb_ETC['z']:.4f}")

SNRdb from TCA_vec_old:
D1: 6.0647, D2: 3.1530, D3: 0.8803

SNRdb from TCA_vec:
D1: 6.0647, D2: 3.1530, D3: 0.8803

SNRdb from ETC:
D1: 6.0647, D2: 3.1530, D3: 0.8803


In [10]:
print("R from TCA_vec_old:")
print(f"D1: {R_TCA_old['x'][0, 0]:.4f}, D2: {R_TCA_old['y'][0, 0]:.4f}, D3: {R_TCA_old['z'][0, 0]:.4f}")

print("\nR from TCA_vec:")
print(f"D1: {R_TCA['x'][0]:.4f}, D2: {R_TCA['y'][0]:.4f}, D3: {R_TCA['z'][0]:.4f}")

print("\nR from ETC:")
print(f"D1: {R_ETC['x']:.4f}, D2: {R_ETC['y']:.4f}, D3: {R_ETC['z']:.4f}")

R from TCA_vec_old:
D1: 0.8016, D2: 0.6739, D3: 0.5505

R from TCA_vec:
D1: 0.8016, D2: 0.6739, D3: 0.5505

R from ETC:
D1: 0.8016, D2: 0.6739, D3: 0.5505


In [11]:
print("fMSE from TCA_vec_old:")
print(f"D1: {fMSE_TCA_old['x'][0, 0]:.4f}, D2: {fMSE_TCA_old['y'][0, 0]:.4f}, D3: {fMSE_TCA_old['z'][0, 0]:.4f}")

print("\nfMSE from TCA_vec:")
print(f"D1: {fMSE_TCA['x'][0]:.4f}, D2: {fMSE_TCA['y'][0]:.4f}, D3: {fMSE_TCA['z'][0]:.4f}")

print("\nfMSE from ETC:")
print(f"D1: {fMSE_ETC['x']:.4f}, D2: {fMSE_ETC['y']:.4f}, D3: {fMSE_ETC['z']:.4f}")

fMSE from TCA_vec_old:
D1: 0.1984, D2: 0.3261, D3: 0.4495

fMSE from TCA_vec:
D1: 0.1984, D2: 0.3261, D3: 0.4495

fMSE from ETC:
D1: 0.1984, D2: 0.3261, D3: 0.4495


In [12]:
print("flag1 from TCA_vec_old:")
print(f"D1: {flags_TCA_old['condition_corr'][0, 0]:.4f}, D2: {flags_TCA_old['condition_corr'][0, 0]:.4f}, D3: {flags_TCA_old['condition_corr'][0, 0]:.4f}")

print("\nflag1 from TCA_vec:")
print(f"D1: {flags_TCA['condition_corr'][0]:.4f}, D2: {flags_TCA['condition_corr'][0]:.4f}, D3: {flags_TCA['condition_corr'][0]:.4f}")

print("\nflag1 from ETC:")
print(f"D1: {flags_ETC['condition_corr']:.4f}, D2: {flags_ETC['condition_corr']:.4f}, D3: {flags_ETC['condition_corr']:.4f}")

#======================================================
print("flag2 from TCA_vec_old:")
print(f"D1: {flags_TCA_old['condition_n_valid'][0, 0]:.4f}, D2: {flags_TCA_old['condition_n_valid'][0, 0]:.4f}, D3: {flags_TCA_old['condition_n_valid'][0, 0]:.4f}")

print("\nflag2 from TCA_vec:")
print(f"D1: {flags_TCA['condition_n_valid'][0]:.4f}, D2: {flags_TCA['condition_n_valid'][0]:.4f}, D3: {flags_TCA['condition_n_valid'][0]:.4f}")

print("\nflag2 from ETC:")
print(f"D1: {flags_ETC['condition_n_valid']:.4f}, D2: {flags_ETC['condition_n_valid']:.4f}, D3: {flags_ETC['condition_n_valid']:.4f}")

#======================================================
print("flag3 from TCA_vec_old:")
print(f"D1: {flags_TCA_old['condition_fMSE'][0, 0]:.4f}, D2: {flags_TCA_old['condition_fMSE'][0, 0]:.4f}, D3: {flags_TCA_old['condition_fMSE'][0, 0]:.4f}")

print("\nflag3 from TCA_vec:")
print(f"D1: {flags_TCA['condition_fMSE'][0]:.4f}, D2: {flags_TCA['condition_fMSE'][0]:.4f}, D3: {flags_TCA['condition_fMSE'][0]:.4f}")

print("\nflag3 from ETC:")
print(f"D1: {flags_ETC['condition_fMSE']:.4f}, D2: {flags_ETC['condition_fMSE']:.4f}, D3: {flags_ETC['condition_fMSE']:.4f}")

#======================================================
print("flag4 from TCA_vec_old:")
print(f"D1: {flags_TCA_old['condition_negative_vars_err'][0, 0]:.4f}, D2: {flags_TCA_old['condition_negative_vars_err'][0, 0]:.4f}, D3: {flags_TCA_old['condition_negative_vars_err'][0, 0]:.4f}")

print("\nflag4 from TCA_vec:")
print(f"D1: {flags_TCA['condition_negative_vars_err'][0]:.4f}, D2: {flags_TCA['condition_negative_vars_err'][0]:.4f}, D3: {flags_TCA['condition_negative_vars_err'][0]:.4f}")

print("\nflag4 from ETC:")
print(f"D1: {flags_ETC['condition_negative_vars_err']:.4f}, D2: {flags_ETC['condition_negative_vars_err']:.4f}, D3: {flags_ETC['condition_negative_vars_err']:.4f}")

flag1 from TCA_vec_old:
D1: 0.0000, D2: 0.0000, D3: 0.0000

flag1 from TCA_vec:
D1: 0.0000, D2: 0.0000, D3: 0.0000

flag1 from ETC:
D1: 0.0000, D2: 0.0000, D3: 0.0000
flag2 from TCA_vec_old:
D1: 0.0000, D2: 0.0000, D3: 0.0000

flag2 from TCA_vec:
D1: 0.0000, D2: 0.0000, D3: 0.0000

flag2 from ETC:
D1: 0.0000, D2: 0.0000, D3: 0.0000
flag3 from TCA_vec_old:
D1: 0.0000, D2: 0.0000, D3: 0.0000

flag3 from TCA_vec:
D1: 0.0000, D2: 0.0000, D3: 0.0000

flag3 from ETC:
D1: 0.0000, D2: 0.0000, D3: 0.0000
flag4 from TCA_vec_old:
D1: 0.0000, D2: 0.0000, D3: 0.0000

flag4 from TCA_vec:
D1: 0.0000, D2: 0.0000, D3: 0.0000

flag4 from ETC:
D1: 0.0000, D2: 0.0000, D3: 0.0000


In [13]:
R_ETC

{'x': 0.8016198649021953, 'y': 0.6739298406912139, 'z': 0.5505031959280736}

In [14]:
R_TCA

{'x': array([0.80161986]), 'y': array([0.67392984]), 'z': array([0.5505032])}